In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


# Task: Columns -> Energy & Petrol Consumption, Storage

In [ ]:
base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Ford", "Nissan"]

def get_data(base, maker):
    mpge = []
    mpg = []
    battery = []
    gas_tank = []
    for m in maker:
        page = 1
        while True:
            print(f"Scraping page {page} of {m}")
            maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={m}&srchtyp=newMake&pageno={page}&rowLimit=200"
            response = r.get(maker_url)
            soup = bs(response.content)
                
            link = [x.get("href") for x in soup.select("a[href]") if "sbs&id" in x.get("href")]
            if len(link) == 0:
                break
                
            for l in link:
                if l == "#":
                    continue
                response = r.get(base+l)
                soup = bs(response.content)
                try:
                    type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
                except:
                    print(f"No type: {l}")
                try:
                    if type == "Electricity":
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                        gas_tank.append(np.nan)
                        mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                        mpg.append(np.nan)
                    elif type in ["Regular Gasoline", "Diesel", "Premium Gasoline"]:
                        battery.append(np.nan)
                        gas_tank.append(soup.find_all("td", class_="sbsEconData")[2].getText().split(" ")[0]) #gallons
                        mpge.append(np.nan)
                        mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                    else:
                        battery.append(soup.find_all("td", class_="even sbsCellData")[2].getText().split(" ")[0]) #hrs per 240v
                        gas_tank.append(soup.find_all("td", class_="sbsEconData")[3].getText().split(" ")[0]) #gallons
                        mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                        mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
                except:
                    print(f"Process stopped at {l}")
            page += 1

    print("Creating Dictionary.")
    print("\n")
    for x in [mpge, mpg, battery, gas_tank]:
        print(len(x))
    manufacturer_dict = {"electric_consumption":mpge, "electric_storage":battery,
                        "analogue_consumption":mpg, "analogue_storage":gas_tank}
    print("\n")
    print("Returning Dataframe.")
    return pd.DataFrame(manufacturer_dict)

manufacturer_df_2 = get_data(base_url, makers)
print("Finished")

Scraping page 1 of Chevrolet
Scraping page 2 of Chevrolet


In [185]:
manufacturer_df

,electric_consumption,electric_storage,analogue_consumption,analogue_storage
0,120,7.5,NaN,NaN
1,120,7.5,NaN,NaN
2,119,9.3,NaN,NaN
3,118,10,NaN,NaN
4,118,10,NaN,NaN
...,...,...,...,...
1286,NaN,NaN,15,26.0
1287,NaN,NaN,15,26.0
1288,NaN,NaN,15,26.0
1289,NaN,NaN,15,26.0
